# 前提環境
Fabric のインストール

In [4]:
!pip install fabric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 555.3 kB/s eta 0:00:001m639.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 756.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 1.5 MB/s eta 0:00:001.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 1.5 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 2.2 MB/s eta 0:00:002.1 MB/s eta 0:00:01


## Fabric ローカル実行

In [65]:
from fabric import task
from invoke.context import Context
import shutil

from fabfile import CheckInstallation, tmux_run

@task
def CheckInstallation(c, command):
    result = shutil.which(command)
    print('=== fabfile.main.CheckInstallation()')
    print(f'--> shutil.which: {result}')
    #print(Context().config.debug)
    print(Context().config.load_base_conf_files)
    
    return not result is None


CheckInstallation(Context(), 'rustup')

=== fabfile.main.CheckInstallation()
--> shutil.which: None
<bound method Config.load_base_conf_files of <Config: {'run': {'asynchronous': False, 'disown': False, 'dry': False, 'echo': False, 'echo_stdin': None, 'encoding': None, 'env': {}, 'err_stream': None, 'fallback': True, 'hide': None, 'in_stream': None, 'out_stream': None, 'echo_format': '\x1b{command}\x1b', 'pty': False, 'replace_env': False, 'shell': '/bin/bash', 'warn': False, 'watchers': []}, 'runners': {'local': <class 'invoke.runners.Local'>}, 'sudo': {'password': None, 'prompt': '[sudo] password: ', 'user': None}, 'tasks': {'auto_dash_names': True, 'collection_name': 'tasks', 'dedupe': True, 'executor_class': None, 'ignore_unknown_help': False, 'search_root': None}, 'timeouts': {'command': None}}>>


False

## Rust のインストール

In [1]:
!pwd

/home/tomoyan/my_projects/jupyter-binder/fabric_example


In [15]:
%%file ./fabfile/install_rust.py
from fabric import task
from invoke.watchers import Responder

from . import CheckInstallation, tmux_run

@task
@tmux_run
def Rust_CheckInstalled(c):
    result = CheckInstallation(c, 'rustup')
    return result

@task
@tmux_run
def Rust_Install(c):
    if Rust_CheckInstalled(c):
        print('Rust はインストール済みです😉');
        return False
    
    enterResponder = Responder(
        pattern=r'\n>',
        response='\n'
    )
    result = c.run(
        "curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh",
        echo=True, hide=False, pty=True,
        watchers=[enterResponder])
    result = c.run(
        "$SHELL -l",
        echo=True, hide=False, pty=True,
        watchers=[enterResponder])
    #print(result.stdout.strip())

@task
@tmux_run
def Rust_Uninstall(c):
    if not Rust_CheckInstalled(c):
        print('Rust はインストールされていません🤔');
        return False
    
    yesResponder = Responder(
        pattern=r'Continue\? \(y/N\)',
        response='y\n'
    )
    result = c.run(
        'rustup self uninstall',
        echo=True, pty=True,
        watchers=[yesResponder]
    )


Overwriting ./fabfile/install_rust.py


In [2]:
!fab Rust-CheckInstalled -d

=== fabfile.main.CheckInstallation()
--> shutil.which(rustup): None


In [3]:
!fab --list

Available tasks:

  CheckInstallation      which command
  ConnectTmux
  Mamba-CheckInstalled
  Mamba-Install
  Mamba-Uninstall
  OLS-install
  OLS-uninstall
  Rust-CheckInstalled
  Rust-Install
  Rust-Uninstall



In [ ]:
!fab Rust-Install -d

=== fabfile.main.CheckInstallation()
--> shutil.which(rustup): None
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh
info: downloading installer

Welcome to Rust!

This will download and install the official compiler for the Rust
programming language, and its package manager, Cargo.

Rustup metadata and toolchains will be installed into the Rustup
home directory, located at:

  /home/tomoyan/.rustup

This can be modified with the RUSTUP_HOME environment variable.

The Cargo home directory is located at:

  /home/tomoyan/.cargo

This can be modified with the CARGO_HOME environment variable.

The cargo, rustc, rustup and other commands will be added to
Cargo's bin directory, located at:

  /home/tomoyan/.cargo/bin

This path will then be added to your PATH environment variable by
modifying the profile files located at:

  /home/tomoyan/.profile
  /home/tomoyan/.bash_profile
  /home/tomoyan/.bashrc

You can uninstall at any time with rustup self uninstall and
these changes w

## Rust のアンインストール

In [7]:
!fab Rust-Uninstall -d

=== fabfile.main.CheckInstallation()
--> shutil.which: None
Rust はインストールされていません🤔


# 参考文献
https://docs.pyinvoke.org/en/stable/concepts/watchers.html<br>
https://blog.masu-mi.me/post/2021/02/14/catch-up-fabric2/<br>